In [1]:
from __future__ import print_function
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.utils as vutils
import time
from torch.autograd import Variable
from dataset import get_dataloader
import numpy as np

In [2]:
from dcgan import NetG, NetD

In [3]:
ngpu = 1
nz = 100
ngf = 64
nc = 3
batch_size = 1

# load netG
netG = NetG(ngpu, nz, ngf, nc)
netG.load_state_dict(torch.load('dcgan_out/netG_epoch_10.pth'))

In [4]:
# generate ground-truth noise
noise = torch.FloatTensor(batch_size, nz, 1, 1).normal_(0, 1)
noise = Variable(noise)
noise.data.resize_(batch_size, nz, 1, 1)
noise.data.normal_(0, 1)

for param in netG.parameters():
    param.requires_grad = False
    
# generate ground-truth fake
fake = netG(noise)

In [5]:
# fix fake, and try to find noise_approx
noise_approx = Variable(torch.FloatTensor(batch_size, nz, 1, 1).normal_(0, 1))
noise_approx.requires_grad = True

mse_loss = nn.MSELoss()
optimizer_approx = optim.Adam([noise_approx], lr=0.1, betas=(0.5, 0.999))

for i in range(50):
    fake_approx = netG(noise_approx)
    loss = mse_loss(fake_approx, fake)
    # mse_z = mse_loss(noise_approx, noise)
    print("Loss after {}\t steps is {}".format(i, loss.data[0]))
    
    optimizer_approx.zero_grad()
    loss.backward()
    optimizer_approx.step()

Loss after 0	 steps is 0.350173681974411
Loss after 1	 steps is 0.22768241167068481
Loss after 2	 steps is 0.15123112499713898
Loss after 3	 steps is 0.11117040365934372
Loss after 4	 steps is 0.09148666262626648
Loss after 5	 steps is 0.08174312859773636
Loss after 6	 steps is 0.07539718598127365
Loss after 7	 steps is 0.07017917186021805
Loss after 8	 steps is 0.06603188812732697
Loss after 9	 steps is 0.06259655207395554
Loss after 10	 steps is 0.05956171825528145
Loss after 11	 steps is 0.05643821880221367
Loss after 12	 steps is 0.05331909656524658
Loss after 13	 steps is 0.05074559524655342
Loss after 14	 steps is 0.048578858375549316
Loss after 15	 steps is 0.046721428632736206
Loss after 16	 steps is 0.04515514150261879
Loss after 17	 steps is 0.04377645626664162
Loss after 18	 steps is 0.04258820414543152
Loss after 19	 steps is 0.04149062559008598
Loss after 20	 steps is 0.04047531634569168
Loss after 21	 steps is 0.03951046243309975
Loss after 22	 steps is 0.0385311059653759

fake = netG(noise)
fake_approx = netG(noise_approx)

In [6]:
vutils.save_image(fake.data, 'fake.png', normalize=True)
vutils.save_image(fake_approx.data, 'fake_approx.png', normalize=True)